# Installing packages section

In [4]:
# Installing packages set for without init script
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0   
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

     |████████████████████████████████| 1.8MB 4.8MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 137.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 136.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 132.7 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforio 1.0.1 requires pandas==2.0.0, but you have pandas 2.2.2 which is incompatible.
fosforml 1.0.1 requires cloudpickle==3.0.0, but you have cloudpickle 2.0.0 which is incompatible.
fosforml 1.0.1 requires numpy==1.26.4; python_version > "3.8", but you have numpy 2.0.1 which is incompatible.
fosforml 1.0.1 requires urllib3==2.2.1, but you have urllib3 1.26.19 which is incompatible.
jsonschema-path 0.3.2 requires referencing<0.32.0,>=0.28.0, but you have referencing 0.33.0 which is incompatible.
jupyterlab 3.2.4 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
mosaic-ai-client 1.0.0 requires matplotlib==3.1.1, but you have matplotlib 3.9.0 which is incompatible.
mosaic-ai-client 1.0.0 requires requests-toolbelt==0.9.1, but you have requests-toolbelt 1.0.0 which is incompati

# Importing packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import joblib
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import configparser
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


# Fetching the datasets

In [2]:
from fosforio import snowflake
from fosforio import get_dataframe

snowflake.get_connection(connection_name="ME_AD_SALES_CXN")

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Connection details fetched: {'params': {'READER': {'connectionId': '87881a55-c8db-4d18-afa7-35fdcdf7ecff', 'connectionSources': {'sourceId': '210d73c1-4da7-487d-8a5c-05c8d2025269', 'connectionType': 'SNOWFLAKE', 'sourceName': 'RDBMS', 'enable': 1, 'displayName': 'SNOWFLAKE', 'displayOrder': 5.0}, 'createdBy': 'ms.akshaya@fosfor.com', 'createdOn': '2024-07-17T14:15:29.278+00:00', 'updatedOn': '2024-08-06T08:46:50.736+00:00', 'name': 'ME_AD_SALES_CXN', 'autosyncable': False, 'autopublishable': False, 'frequency': None, 'assetConfigurations': [], 'publishableNodes': [], 'autoPublishableNodes': None, 'isPrivate': False, 'isExpired': False, 'autoSyncFrequency': None, 'application': None, 'sourceDataRefreshFrequency': None, 'defaultDb': 'ME_DB', 'defaultSchema': 'ME_AD_SALES_SCHEMA', 'recommendedPrivileges'

In [3]:
df_all = get_dataframe("DF_ALL")
df_opt = get_dataframe("DF_OPT")

Reading dataframe from snowflake native connector
project_id: 32bc8d89-e57a-4d61-9779-4f165eaf05aa
Connection details fetched: {'params': {'READER': {'schema': 'ME_AD_SALES_SCHEMA', 'cloudPlatform': 'gcp', 'role': '"AKSHAYA"', 'clientId': None, 'tokenUrl': None, 'scopeUrl': None, 'type': 'RDBMS', 'wareHouse': 'FOSFOR_SOLUTIONS_WH', 'accountId': 'ug94937', 'privateKey': None, 'password': 'Password@2023', 'database': 'ME_DB', 'tables': 'DF_ALL', 'sub_type': 'SNOWFLAKE', 'subType': 'SNOWFLAKE', 'clientSecret': None, 'authenticationType': 'dbPassword', 'rowCount': 10, 'region': 'us-east4', 'user': 'Akshaya', 'applicationName': 'FOSFOR', 'passPhrase': None}}, 'url': 'http://connectors-backend-service/connectors/v1/connectors/metadata', 'access': True, 'errorMsg': None, 'values': {}}
Exception occurred in get_dataframe: No module named 'cryptography.__about__'
Reading dataframe from snowflake native connector
project_id: 32bc8d89-e57a-4d61-9779-4f165eaf05aa
Connection details fetched: {'para